In [1]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

# Planejamento de Geração de Energia

### Descrição do Problema

A demanda de energia elétrica diária de uma determinada cidade, estratificada pelas horas do dia, é apresentada na Tabela 1. Há três tipos de geradores: 12 unidades do gerador do tipo 1, 10 do tipo 2 e 5 do tipo 3. Cada gerador é capaz de produzir energia em um determinado intervalo, compreendendo um valor mínimo de geração e um valor máximo. Para cada gerador, há um custo horário de geração de energia, por MW, quando o gerador opera no seu mínimo operacional. Adicionalmente, há um custo horário adicional, por MW gerado além do mínimo operacional. A operação de ligar um gerador que se encontra desligado também incorre em um custo (custo de setup), de natureza fixa. Todos estes custos são apresentados na Tabela 2.

Além de produzir em quantidade suficiente para atender à demanda de cada intervalo do dia, o conjunto de geradores em operação a qualquer momento do dia deve ter capacidade operacional suficiente para atender um acréscimo de 15% na demanda, apenas ajustando o ponto de operação de cada gerador (sem ligar geradores adicionais no mesmo intervalo de tempo do dia), de forma que a operação de cada gerador ligado ainda permaneça dentro dos limites estabelecidos.

[Mais detalhes da modelagem](exercicio-geracao-energia-eletrica.pdf)


## Parâmetros para a modelagem

In [2]:
maxt = 5  # Número de períodos do dia
maxg = 3  # Número de tipos de geradores
T = list(range(0, maxt)) # Conjunto de índices de períodos do dia
G = list(range(0, maxg)) # Conjunto de tipos de geradores
l = [6, 3, 6, 3, 6]  # Duração em horas de cada período do dia. Duração de T[i] = l[i]
n = [12, 10, 5] # Quantidade disponível de cada tipo de gerador
mn = [850, 1250, 1500] # Potência operacional mínima de cada gerador
mx = [2000, 1750, 4000] # Potência operacional máxima de cada gerador
c = [1000, 2600, 3000] # Custo da geração por MW. Custo de g[i] = c[i]
v = [1020, 2652, 3060] # Custo adicional de geração por MW. Custo de g[i] = c[i]
s = [2000, 1000, 500] # Custo de ligar o compressor pelo tipo. Custo de g[i] = c[i]
p = 0.15 # Percentual de segurança operacional (reserva operacional, %)
d = [15000, 30000, 25000, 40000, 27000] # Demanda de geração para cada intervalo. Demanda de T[i] = d[i]

## Variáveis de Decisão

In [3]:

Modelo = ConcreteModel()

# Quantidade de geradores do tipo g ∈ G que estão ligados e funcionando no período do dia t ∈ T
Modelo.z = Var(G, T, within=NonNegativeIntegers)  

# Quantidade de geradores que foram ligados em t que não estavam ligados no período anterior t - 1
Modelo.w = Var(G, T, within=NonNegativeIntegers)  

# Quantidade total de energia produzida no período t pelos geradores do tipo g que estiverem em funcionamento
Modelo.x = Var(G, T, within=NonNegativeIntegers) 

## Parcelas da função objetivo

In [4]:
# Custo fixo de ligar as unidades do tipo g em t
custo_fixo = sum(sum(s[g] * Modelo.w[g, t] for t in T) for g in G)

# Custo de produzir ao mínimo nível operacional, o conjunto de geradores que for ligado.
custo_minimo = sum(sum(mn[g] * c[g] * l[t] * Modelo.z[g, t] for t in T) for g in G)

# Custo de produzir além do mínimo operacional, o conjunto de geradores que for ligado
custo_adicional = sum(sum(v[g] * l[t] * (Modelo.x[g, t] - mn[g] * Modelo.z[g, t]) for t in T) for g in G)

## Definindo as restrições

In [5]:
Modelo.restricoes = ConstraintList()


# Atendimento das faixas operacionais dos compressores
for g in G:
    for t in T:
        Modelo.restricoes.add(Modelo.x[g, t] >= mn[g] * Modelo.z[g, t])
        Modelo.restricoes.add(Modelo.x[g, t] <= mx[g] * Modelo.z[g, t])

# Limites superiores no n´umero de geradores ligados:
for g in G:
    for t in T:
        Modelo.restricoes.add(Modelo.z[g, t] <= n[g])

# Atendimento da demanda
for t in T:
    Modelo.restricoes.add(sum(Modelo.x[g, t] for g in G) >= d[t])

# Flexibilidade operacional
for t in T:
    Modelo.restricoes.add(sum(mx[g] * Modelo.z[g, t] for g in G) >=  d[t] * (1+p))

# Acoplamento entre variáveis que indicam o número total de geradores ligados de um tipo g ∈ G 
# e quantos daquele tipo que foram ligados no período t ∈ T
for t in T[1:]:
    for g in G:
        Modelo.restricoes.add(Modelo.w[g, t] >= Modelo.z[g, t] - Modelo.z[g, (t-1)])
        Modelo.restricoes.add(Modelo.w[g, 0] >= Modelo.z[g, 0] - Modelo.z[g, (maxt-1)])
    

## Definindo o objetivo 
### (Minimizar)

In [6]:
Modelo.objetivo = Objective(expr=(custo_fixo + custo_minimo + custo_adicional), sense=minimize)

## Resolvendo:

In [7]:
solver = SolverFactory('glpk')
solver.solve(Modelo, tee=True)

custo_total = value(Modelo.objetivo)

print("\n\n\n")
print("----------------------- RESULTADO -----------------------")
print(f"Custo total: ${custo_total:.2f} USD \n")

# Total de geradores operantes
print("Total de geradores operantes:")
print("Instantes |\t 1 |\t 2 |\t 3 |\t 4 |\t 5")
print("--------------------------------------------------")
for g in G:
    print(f"Tipo {g+1}   |\t", " |\t".join(f"{int(value(Modelo.z[g, t])):02d}" for t in T))

# Total de geradores que foram ligados
print("\nTotal de geradores que foram ligados:")
print("Instantes |\t 1 |\t 2 |\t 3 |\t 4 |\t 5")
print("--------------------------------------------------")
for g in G:
    print(f"Tipo {g+1}   |\t", " |\t".join(f"{int(value(Modelo.w[g, t]))}" for t in T))

# Total de demanda atendida em cada período do dia
for t in T:
    total_demanda = sum(value(Modelo.x[g, t]) for g in G)
    print(f"\nTotal de demanda atendida no período do dia t={t+1}: {total_demanda:.2f}")


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\CAU~1\AppData\Local\Temp\tmp_74yk18w.glpk.raw --wglp C:\Users\CAU~1\AppData\Local\Temp\tmp5brcvwhh.glpk.glp
 --cpxlp C:\Users\CAU~1\AppData\Local\Temp\tmpfjws9sui.pyomo.lp
Reading problem data from 'C:\Users\CAU~1\AppData\Local\Temp\tmpfjws9sui.pyomo.lp'...
79 rows, 45 columns, 177 non-zeros
45 integer variables, none of which are binary
559 lines were read
Writing problem data to 'C:\Users\CAU~1\AppData\Local\Temp\tmp5brcvwhh.glpk.glp'...
473 lines were written
GLPK Integer Optimizer, v4.65
79 rows, 45 columns, 177 non-zeros
45 integer variables, none of which are binary
Preprocessing...
64 rows, 45 columns, 162 non-zeros
45 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  4.000e+03  ratio =  4.000e+03
GM: min|aij| =  7.825e-01  max|aij| =  1.278e+00  ratio =  1.633e+00
EQ: min|aij| =  6.124e-01  max|aij| =  1.000e+00  ratio =  1.633e+00
2N: min|